# Data

At a minimum, the following data is required to construct a model to estimate accident severity
- Collision statistics that include a severity measure
- Location information or road characteristics for each of the collisions to allow extraposation to other similar sections of road
- Road surface condition and other environmental features that relate to each of the collisions

The viability of producing an accurate collision severity model will utilise the collison data from the Seattle Police Department accessible via the following link: 
[Seattle Collision Data](https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv).

A description of the dataset can be found via the following link: 
[Seattle Collision Metadata](https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Metadata.pdf).

The remainder of this section contains an assessment of the candidate data set and and explaination of the data elements that are used to construct the model.

## Initial Assessment
Firstly, the data is loaded for evaluation and some basic analysis is performed to get an overview of the contents of the dataset.

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [3]:
collisions_data_path = "https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv"
df = pd.read_csv(collisions_data_path, low_memory=False)

Visually inspect a subset of the dataset to confirm that it has loaded and to confirm the amount of data and data types available.

In [4]:
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [5]:
df.shape

(194673, 38)

In [6]:
print(df.dtypes)

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

DECISION: From the Metadata descriptions and inspecting the output of the head function, the following columns containing identifier and key values will not be investigated:
- OBJECTID
- INCKEY
- COLDETKEY
- REPORTNO

Perform some high level statistical analysis of the data to aid in narrowing down relavant features.

In [7]:
firstSet = ['SEVERITYCODE','X','Y','STATUS','ADDRTYPE','INTKEY','LOCATION','EXCEPTRSNCODE','EXCEPTRSNDESC',
               'SEVERITYCODE.1','SEVERITYDESC','COLLISIONTYPE','PERSONCOUNT','PEDCOUNT','PEDCYLCOUNT','VEHCOUNT']
secondSet = ['INCDATE','INCDTTM','JUNCTIONTYPE','SDOT_COLCODE','SDOT_COLDESC','INATTENTIONIND','UNDERINFL','WEATHER','ROADCOND','LIGHTCOND','PEDROWNOTGRNT','SDOTCOLNUM',
             'SPEEDING','ST_COLCODE','ST_COLDESC','SEGLANEKEY','CROSSWALKKEY','HITPARKEDCAR']

In [8]:
df[firstSet].describe(include='all')

,SEVERITYCODE,X,Y,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT
count,194673.000000,189339.000000,189339.000000,194673,192747,65070.000000,191996,84811,5638,194673.000000,194673,189769,194673.000000,194673.000000,194673.000000,194673.000000
unique,NaN,NaN,NaN,2,3,NaN,24102,2,1,NaN,2,10,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,Matched,Block,NaN,BATTERY ST TUNNEL NB BETWEEN ALASKAN WY VI NB ...,,"Not Enough Information, or Insufficient Locati...",NaN,Property Damage Only Collision,Parked Car,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,189786,126926,NaN,276,79173,5638,NaN,136485,47987,NaN,NaN,NaN,NaN
mean,1.298901,-122.330518,47.619543,NaN,NaN,37558.450576,NaN,NaN,NaN,1.298901,NaN,NaN,2.444427,0.037139,0.028391,1.920780
std,0.457778,0.029976,0.056157,NaN,NaN,51745.990273,NaN,NaN,NaN,0.457778,NaN,NaN,1.345929,0.198150,0.167413,0.631047
min,1.000000,-122.419091,47.495573,NaN,NaN,23807.000000,NaN,NaN,NaN,1.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000
25%,1.000000,-122.348673,47.575956,NaN,NaN,28667.000000,NaN,NaN,NaN,1.000000,NaN,NaN,2.000000,0.000000,0.000000,2.000000
50%,1.000000,-122.330224,47.615369,NaN,NaN,29973.000000,NaN,NaN,NaN,1.000000,NaN,NaN,2.000000,0.000000,0.000000,2.000000
75%,2.000000,-122.311937,47.663664,NaN,NaN,33973.000000,NaN,NaN,NaN,2.000000,NaN,NaN,3.000000,0.000000,0.000000,2.000000


In [9]:
df[secondSet].describe(include='all')

,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
count,194673,194673,188344,194673.000000,194673,29805,189789,189592,189661,189503,4667,1.149360e+05,9333,194655,189769,194673.000000,1.946730e+05,194673
unique,5985,162058,7,NaN,39,1,4,11,9,9,1,NaN,1,63,62,NaN,NaN,2
top,2006/11/02 00:00:00+00,11/2/2006,Mid-Block (not related to intersection),NaN,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",Y,N,Clear,Dry,Daylight,Y,NaN,Y,32,One parked--one moving,NaN,NaN,N
freq,96,96,89800,NaN,85209,29805,100274,111135,124510,116137,4667,NaN,9333,44421,44421,NaN,NaN,187457
mean,NaN,NaN,NaN,13.867768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.972521e+06,NaN,NaN,NaN,269.401114,9.782452e+03,NaN
std,NaN,NaN,NaN,6.868755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.553533e+06,NaN,NaN,NaN,3315.776055,7.226926e+04,NaN
min,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.007024e+06,NaN,NaN,NaN,0.000000,0.000000e+00,NaN
25%,NaN,NaN,NaN,11.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.040015e+06,NaN,NaN,NaN,0.000000,0.000000e+00,NaN
50%,NaN,NaN,NaN,13.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.023022e+06,NaN,NaN,NaN,0.000000,0.000000e+00,NaN
75%,NaN,NaN,NaN,14.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.015501e+07,NaN,NaN,NaN,0.000000,0.000000e+00,NaN


In [10]:
df.corr()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,INTKEY,SEVERITYCODE.1,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,SDOT_COLCODE,SDOTCOLNUM,SEGLANEKEY,CROSSWALKKEY
SEVERITYCODE,1.000000,0.010309,0.017737,0.020131,0.022065,0.022079,0.006553,1.000000,0.130949,0.246338,0.214218,-0.054686,0.188905,0.004226,0.104276,0.175093
X,0.010309,1.000000,-0.160262,0.009956,0.010309,0.010300,0.120754,0.010309,0.012887,0.011304,-0.001752,-0.012168,0.010904,-0.001016,-0.001618,0.013586
Y,0.017737,-0.160262,1.000000,-0.023848,-0.027396,-0.027415,-0.114935,0.017737,-0.013850,0.010178,0.026304,0.017058,-0.019694,-0.006958,0.004618,0.009508
OBJECTID,0.020131,0.009956,-0.023848,1.000000,0.946383,0.945837,0.046929,0.020131,-0.062333,0.024604,0.034432,-0.094280,-0.037094,0.969276,0.028076,0.056046
INCKEY,0.022065,0.010309,-0.027396,0.946383,1.000000,0.999996,0.048524,0.022065,-0.061500,0.024918,0.031342,-0.107528,-0.027617,0.990571,0.019701,0.048179
COLDETKEY,0.022079,0.010300,-0.027415,0.945837,0.999996,1.000000,0.048499,0.022079,-0.061403,0.024914,0.031296,-0.107598,-0.027461,0.990571,0.019586,0.048063
INTKEY,0.006553,0.120754,-0.114935,0.046929,0.048524,0.048499,1.000000,0.006553,0.001886,-0.004784,0.000531,-0.012929,0.007114,0.032604,-0.010510,0.018420
SEVERITYCODE.1,1.000000,0.010309,0.017737,0.020131,0.022065,0.022079,0.006553,1.000000,0.130949,0.246338,0.214218,-0.054686,0.188905,0.004226,0.104276,0.175093
PERSONCOUNT,0.130949,0.012887,-0.013850,-0.062333,-0.061500,-0.061403,0.001886,0.130949,1.000000,-0.023464,-0.038809,0.380523,-0.128960,0.011784,-0.021383,-0.032258
PEDCOUNT,0.246338,0.011304,0.010178,0.024604,0.024918,0.024914,-0.004784,0.246338,-0.023464,1.000000,-0.016920,-0.261285,0.260393,0.021461,0.001810,0.565326


## Geospacial View
A plot Seattle with an overview of property damage (yellow) and injury (red) was produced to see if location was significant in the outcome of an incident. 

In [11]:
limit = 5000
df_collisions = df.iloc[0:limit, :]

collisions = folium.map.FeatureGroup()

for lat, lng, severity in zip(df_collisions.Y, df_collisions.X, df_collisions.SEVERITYCODE):
    if not math.isnan(lat) and not math.isnan(lng):
        if severity == 1:
            color='yellow'
        else:
            color='red'
        collisions.add_child(
            folium.features.CircleMarker(
                [lat, lng],
                radius=5, # define how big you want the circle markers to be
                color=color,
                fill=True,
                fill_color='blue',
                fill_opacity=0.6
            )
        )
        
        

# define a map centered around Seattle
collision_map = folium.Map(location=[47.6062, -122.3321], zoom_start=12)
collision_map.add_child(collisions)

The overview of the first five thousand collisions does not show an obvious bias based on location so will not be used in modelling.

## Analysing Discrete Features
A number of the colums contain discrete values which merit further investigation.  The value_counts method is used to provide a quick overview of the data.

In [12]:
df['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [13]:
df['STATUS'].value_counts()

Matched      189786
Unmatched      4887
Name: STATUS, dtype: int64

DECISION: STATUS will not be used for the prediciton model

In [14]:
df['ADDRTYPE'].value_counts()

Block           126926
Intersection     65070
Alley              751
Name: ADDRTYPE, dtype: int64

DECISION: ADDRTYPE appears useful for generic prediction along routes as Block, Intersection and Alley are relatively easy to determine for other road networks.

In [15]:
df['LOCATION'].value_counts()

BATTERY ST TUNNEL NB BETWEEN ALASKAN WY VI NB AND AURORA AVE N                          276
BATTERY ST TUNNEL SB BETWEEN AURORA AVE N AND ALASKAN WY VI SB                          271
N NORTHGATE WAY BETWEEN MERIDIAN AVE N AND CORLISS AVE N                                265
AURORA AVE N BETWEEN N 117TH PL AND N 125TH ST                                          254
6TH AVE AND JAMES ST                                                                    252
AURORA AVE N BETWEEN N 130TH ST AND N 135TH ST                                          239
ALASKAN WY VI NB BETWEEN S ROYAL BROUGHAM WAY ON RP AND SENECA ST OFF RP                238
RAINIER AVE S BETWEEN S BAYVIEW ST AND S MCCLELLAN ST                                   231
ALASKAN WY VI SB BETWEEN COLUMBIA ST ON RP AND ALASKAN WY VI SB EFR OFF RP              212
WEST SEATTLE BR EB BETWEEN ALASKAN WY VI NB ON RP AND DELRIDGE-W SEATTLE BR EB ON RP    212
AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N                                      

DECISION: LOCATION appears too specific for a general purpose prediction.

In [16]:
df['EXCEPTRSNCODE'].value_counts()

       79173
NEI     5638
Name: EXCEPTRSNCODE, dtype: int64

In [17]:
df['EXCEPTRSNDESC'].value_counts()

Not Enough Information, or Insufficient Location Information    5638
Name: EXCEPTRSNDESC, dtype: int64

DECISION: EXCEPTRSNCODE and EXCEPTRSNDECS may be a useful detail to identify and drop incomplete information.

In [18]:
df['SEVERITYCODE.1'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE.1, dtype: int64

In [19]:
df['SEVERITYDESC'].value_counts()

Property Damage Only Collision    136485
Injury Collision                   58188
Name: SEVERITYDESC, dtype: int64

DECISION: SEVERITYCODE.1 and SEVERITYDESC appear to be duplicates of the SEVERITY column and will not be evaluated further.

In [20]:
df['COLLISIONTYPE'].value_counts()

Parked Car    47987
Angles        34674
Rear Ended    34090
Other         23703
Sideswipe     18609
Left Turn     13703
Pedestrian     6608
Cycles         5415
Right Turn     2956
Head On        2024
Name: COLLISIONTYPE, dtype: int64

DECISION: COLLISIONTYPE is unlikely to be useful as a prediction of the collion type may be difficult to predict but it may be analysed further during modelling as it may be correlated with other features useful for determining routes (e.g. Left Turn at an intersection may be more likely to result in an injury which may require an alternate route).

In [21]:
df['PERSONCOUNT'].value_counts()

2     114231
3      35553
4      14660
1      13154
5       6584
0       5544
6       2702
7       1131
8        533
9        216
10       128
11        56
12        33
13        21
14        19
15        11
17        11
16         8
44         6
18         6
20         6
25         6
19         5
26         4
22         4
27         3
28         3
29         3
47         3
32         3
34         3
37         3
23         2
21         2
24         2
30         2
36         2
57         1
31         1
35         1
39         1
41         1
43         1
48         1
53         1
54         1
81         1
Name: PERSONCOUNT, dtype: int64

In [22]:
df['PEDCOUNT'].value_counts()

0    187734
1      6685
2       226
3        22
4         4
6         1
5         1
Name: PEDCOUNT, dtype: int64

In [23]:
df['PEDCYLCOUNT'].value_counts()

0    189189
1      5441
2        43
Name: PEDCYLCOUNT, dtype: int64

In [24]:
df['PEDROWNOTGRNT'].value_counts()

Y    4667
Name: PEDROWNOTGRNT, dtype: int64

In [25]:
df['VEHCOUNT'].value_counts()

2     147650
1      25748
3      13010
0       5085
4       2426
5        529
6        146
7         46
8         15
9          9
11         6
10         2
12         1
Name: VEHCOUNT, dtype: int64

DECISION: The counts will not be further evalutated as they are a consequence of a collision and are unlikely to predict severity.

In [26]:
df['INCDATE'].value_counts()

2006/11/02 00:00:00+00    96
2008/10/03 00:00:00+00    92
2005/05/18 00:00:00+00    84
2006/01/13 00:00:00+00    83
2005/11/05 00:00:00+00    83
2008/10/31 00:00:00+00    82
2005/04/29 00:00:00+00    76
2005/04/15 00:00:00+00    75
2007/10/19 00:00:00+00    74
2004/12/04 00:00:00+00    74
2005/10/28 00:00:00+00    73
2007/07/20 00:00:00+00    73
2006/06/01 00:00:00+00    73
2016/10/13 00:00:00+00    73
2007/11/15 00:00:00+00    70
2010/11/22 00:00:00+00    70
2006/10/18 00:00:00+00    70
2006/11/04 00:00:00+00    70
2006/11/22 00:00:00+00    69
2005/11/04 00:00:00+00    69
2006/11/06 00:00:00+00    68
2006/11/10 00:00:00+00    68
2005/12/10 00:00:00+00    68
2007/01/05 00:00:00+00    68
2006/05/05 00:00:00+00    68
2010/10/09 00:00:00+00    68
2006/11/21 00:00:00+00    68
2005/11/11 00:00:00+00    68
2006/04/08 00:00:00+00    68
2006/02/24 00:00:00+00    67
                          ..
2020/03/21 00:00:00+00     6
2020/04/07 00:00:00+00     6
2020/04/29 00:00:00+00     6
2020/04/27 00:

DECISION: INCDATE may be evaluated further to determine whether season or month can improve the accuracy of the model beyond just weather, road condition or light. 

In [27]:
df['ST_COLDESC'].value_counts()

One parked--one moving                                                                   44421
Entering at angle                                                                        34674
From same direction - both going straight - one stopped - rear-end                       25771
Fixed object                                                                             13554
From same direction - both going straight - both moving - sideswipe                      12777
From opposite direction - one left turn - one straight                                   10324
From same direction - both going straight - both moving - rear-end                        7629
Vehicle - Pedalcyclist                                                                    4701
From same direction - all others                                                          4537
From same direction - one left turn - one straight                                        3093
From same direction - one right turn - one straigh

In [28]:
df['HITPARKEDCAR'].value_counts()

N    187457
Y      7216
Name: HITPARKEDCAR, dtype: int64

In [29]:
df['INCDTTM'].value_counts()

11/2/2006                 96
10/3/2008                 91
11/5/2005                 83
12/4/2004                 74
6/1/2006                  73
11/4/2006                 70
11/4/2005                 69
5/5/2006                  68
1/5/2007                  68
4/8/2006                  68
11/6/2006                 68
11/1/2005                 67
11/1/2008                 67
3/8/2006                  65
10/6/2006                 65
1/9/2006                  64
1/2/2004                  64
11/3/2006                 64
10/6/2005                 62
8/6/2004                  62
7/8/2005                  61
6/9/2005                  61
10/2/2007                 60
4/3/2006                  60
11/6/2008                 60
5/6/2009                  60
2/5/2008                  59
2/2/2006                  59
6/1/2007                  59
1/3/2006                  58
                          ..
6/3/2018 10:45:00 AM       1
10/24/2008 12:00:00 PM     1
7/15/2010 5:00:00 PM       1
7/5/2011 8:44:

DECISION: INCDTTM may be used in place of INCDATE if date based improvements are required.

In [30]:
df['JUNCTIONTYPE'].value_counts()

Mid-Block (not related to intersection)              89800
At Intersection (intersection related)               62810
Mid-Block (but intersection related)                 22790
Driveway Junction                                    10671
At Intersection (but not related to intersection)     2098
Ramp Junction                                          166
Unknown                                                  9
Name: JUNCTIONTYPE, dtype: int64

DECISION: JUNCTIONTYPE may be used if there is correlation with severity.

In [31]:
df['SDOT_COLDESC'].value_counts()

MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END AT ANGLE          85209
MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END                    54299
MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE SIDESWIPE          9928
NOT ENOUGH INFORMATION / NOT APPLICABLE                          9787
MOTOR VEHICLE RAN OFF ROAD - HIT FIXED OBJECT                    8856
MOTOR VEHCILE STRUCK PEDESTRIAN                                  6518
MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE AT ANGLE           5852
MOTOR VEHICLE STRUCK OBJECT IN ROAD                              4741
MOTOR VEHICLE STRUCK PEDALCYCLIST, FRONT END AT ANGLE            3104
MOTOR VEHICLE STRUCK MOTOR VEHICLE, RIGHT SIDE SIDESWIPE         1604
MOTOR VEHICLE STRUCK MOTOR VEHICLE, RIGHT SIDE AT ANGLE          1440
PEDALCYCLIST STRUCK MOTOR VEHICLE FRONT END AT ANGLE             1312
MOTOR VEHICLE OVERTURNED IN ROAD                                  479
MOTOR VEHICLE STRUCK PEDALCYCLIST, REAR END                       181
PEDALCYCLIST STRUCK 

DECISION: SDOT_COLDESC will not be used for predicting severity.

In [32]:
df['INATTENTIONIND'].value_counts()

Y    29805
Name: INATTENTIONIND, dtype: int64

In [33]:
df['UNDERINFL'].value_counts()

N    100274
0     80394
Y      5126
1      3995
Name: UNDERINFL, dtype: int64

DECISION: INATTENTIONIND and UNDERINFL will not be used for predicting severity as they will not be an input into route planning.

In [34]:
df['WEATHER'].value_counts()

Clear                       111135
Raining                      33145
Overcast                     27714
Unknown                      15091
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [35]:
df['ROADCOND'].value_counts()

Dry               124510
Wet                47474
Unknown            15078
Ice                 1209
Snow/Slush          1004
Other                132
Standing Water       115
Sand/Mud/Dirt         75
Oil                   64
Name: ROADCOND, dtype: int64

In [36]:
df['LIGHTCOND'].value_counts()

Daylight                    116137
Dark - Street Lights On      48507
Unknown                      13473
Dusk                          5902
Dawn                          2502
Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Other                          235
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

DECISION: WEATHER, ROADCOND and LIGHTCOND are likely to be useful and will require further analysis.
DECISION: The remaining columns below will not be used for model development.

In [37]:
df['PEDROWNOTGRNT'].value_counts()

Y    4667
Name: PEDROWNOTGRNT, dtype: int64

In [38]:
df['SDOTCOLNUM'].value_counts()

4116034.0     2
11200007.0    2
4112025.0     2
4116048.0     2
5036003.0     1
12030005.0    1
5036023.0     1
10161007.0    1
4028036.0     1
7087008.0     1
12004052.0    1
10161018.0    1
12027022.0    1
5036011.0     1
10342027.0    1
11161009.0    1
4028033.0     1
6078022.0     1
10278010.0    1
6078010.0     1
7087039.0     1
7219004.0     1
10209035.0    1
6078007.0     1
5118001.0     1
11210029.0    1
6316024.0     1
8209029.0     1
8161007.0     1
10204033.0    1
             ..
5051014.0     1
11228013.0    1
11213021.0    1
12250006.0    1
12236001.0    1
11358002.0    1
8267010.0     1
8152044.0     1
9212041.0     1
11212026.0    1
9070044.0     1
8152036.0     1
6262034.0     1
9002013.0     1
11172019.0    1
9278028.0     1
8337019.0     1
6176011.0     1
9097020.0     1
8337011.0     1
6277017.0     1
6227002.0     1
8337005.0     1
6277012.0     1
12157032.0    1
6262043.0     1
6162018.0     1
6277003.0     1
8267022.0     1
5071015.0     1
Name: SDOTCOLNUM, Length

In [39]:
df['SPEEDING'].value_counts()

Y    9333
Name: SPEEDING, dtype: int64

In [40]:
df['SDOT_COLCODE'].value_counts()

11    85209
14    54299
16     9928
0      9787
28     8856
24     6518
13     5852
26     4741
18     3104
15     1604
12     1440
51     1312
29      479
21      181
56      180
27      166
54      139
23      124
48      107
31      104
25      102
34       93
64       75
69       69
33       53
55       50
66       23
22       17
32       12
53        9
44        8
61        7
35        6
58        5
68        4
36        4
46        3
52        2
47        1
Name: SDOT_COLCODE, dtype: int64

In [41]:
df['ST_COLCODE'].value_counts()

32    44421
10    34674
14    25771
50    13554
11    12777
28    10324
13     7629
       4886
45     4701
23     4537
15     3093
16     2956
0      2882
20     2846
12     2435
22     2274
2      2178
21     1617
30     1302
1      1201
71     1184
26     1039
81      835
52      815
19      720
24      590
5       416
51      371
74      343
29      286
      ...  
73      167
25      132
4       111
57      108
40      103
84       94
83       86
72       73
41       57
64       50
31       47
82       35
56       34
48       32
53       26
8        23
7        18
65       11
42       11
66       11
17        9
67        9
88        8
54        7
18        5
87        2
43        2
85        1
49        1
60        1
Name: ST_COLCODE, Length: 63, dtype: int64

In [42]:
df['ST_COLDESC'].value_counts()

One parked--one moving                                                                   44421
Entering at angle                                                                        34674
From same direction - both going straight - one stopped - rear-end                       25771
Fixed object                                                                             13554
From same direction - both going straight - both moving - sideswipe                      12777
From opposite direction - one left turn - one straight                                   10324
From same direction - both going straight - both moving - rear-end                        7629
Vehicle - Pedalcyclist                                                                    4701
From same direction - all others                                                          4537
From same direction - one left turn - one straight                                        3093
From same direction - one right turn - one straigh

In [43]:
df['SEGLANEKEY'].value_counts()

0         191907
6532          19
6078          16
12162         15
10336         14
10342         13
8985          12
10354         10
10420         10
8816          10
12179         10
10368          9
10590          8
8995           8
10773          8
42777          7
10566          7
12941          7
10374          7
12649          6
8990           6
8240           6
12035          6
10532          6
42166          6
23507          6
6322           6
9002           6
10408          6
2426           6
           ...  
4467           1
6854           1
9153           1
13251          1
13891          1
35669          1
19149          1
18762          1
32460          1
8647           1
18890          1
6848           1
41943          1
15428          1
7360           1
38097          1
23674          1
11718          1
34771          1
25288          1
6215           1
41040          1
10433          1
6343           1
37987          1
35157          1
10817          1
15043         

In [44]:
df['CROSSWALKKEY'].value_counts()

0         190862
523609        17
520838        15
525567        13
521707        10
523699        10
523148         9
521863         9
521604         9
523735         9
524265         9
522891         9
522264         8
524689         8
525659         8
521040         8
523987         8
520855         8
523109         8
524029         8
522108         8
522377         8
524178         8
525644         8
521845         7
524221         7
523172         7
525079         7
521865         7
523707         7
           ...  
523320         1
525639         1
523704         1
616043         1
523578         1
29899          1
525381         1
521275         1
522811         1
522939         1
31563          1
523195         1
525508         1
521530         1
521658         1
26056          1
522373         1
523963         1
524091         1
524219         1
522298         1
522426         1
37207          1
524997         1
619243         1
521019         1
630862         1
25545         

In [45]:
df['HITPARKEDCAR'].value_counts()

N    187457
Y      7216
Name: HITPARKEDCAR, dtype: int64

## Further Analysis

This section captures a quick analysis of the candidate data for inclusion in the model.

Firstly, a quick assessment of the relationship between SEVERITYCODE and each of the candidates as well as some basic statistics (count, average and standard deviation).  The mean and standard deviation are relevant as severity code is either 1 or 2, so a mean closer to 2 indicates more likelyhood of an injury.

Further analysis of EXCEPTRSNCODE, EXCEPTRSNDECS and INCDATE may be performed in subsequent phases of the project.

### ADDRTYPE

In [46]:
df.groupby(['ADDRTYPE','SEVERITYCODE']).size()

ADDRTYPE      SEVERITYCODE
Alley         1                 669
              2                  82
Block         1               96830
              2               30096
Intersection  1               37251
              2               27819
dtype: int64

In [47]:
df.groupby('ADDRTYPE').agg({'SEVERITYCODE': ['count', 'mean', 'std']})

SEVERITYCODE                    
                    count      mean       std
ADDRTYPE                                     
Alley                 751  1.109188  0.312082
Block              126926  1.237115  0.425315
Intersection        65070  1.427524  0.494723

DECISION: ADDRTYPE will be used in the model as there is a significant severity ratio differance betwen Allay, Block and Intersection.

### JUNCTIONTYPE

In [48]:
df.groupby(['JUNCTIONTYPE','SEVERITYCODE']).size()

JUNCTIONTYPE                                       SEVERITYCODE
At Intersection (but not related to intersection)  1                1475
                                                   2                 623
At Intersection (intersection related)             1               35636
                                                   2               27174
Driveway Junction                                  1                7437
                                                   2                3234
Mid-Block (but intersection related)               1               15493
                                                   2                7297
Mid-Block (not related to intersection)            1               70396
                                                   2               19404
Ramp Junction                                      1                 112
                                                   2                  54
Unknown                                            1        

In [49]:
df.groupby('JUNCTIONTYPE').agg({'SEVERITYCODE': ['count', 'mean', 'std']})

SEVERITYCODE            \
                                                         count      mean   
JUNCTIONTYPE                                                               
At Intersection (but not related to intersection)         2098  1.296949   
At Intersection (intersection related)                   62810  1.432638   
Driveway Junction                                        10671  1.303064   
Mid-Block (but intersection related)                     22790  1.320184   
Mid-Block (not related to intersection)                  89800  1.216080   
Ramp Junction                                              166  1.325301   
Unknown                                                      9  1.222222   

                                                             
                                                        std  
JUNCTIONTYPE                                                 
At Intersection (but not related to intersection)  0.457023  
At Intersection (intersection related)             0.495446  
Driveway Junction                                  0.459604  
Mid-Block (but intersection related)               0.466557  
Mid-Block (not related to intersection)            0.411572  
Ramp Junction                                      0.469905  
Unknown                                            0.440959

DECISION: JUNCTIONTYPE may be added to the model after the first iteration if accurancy needs to be improved because it looks like it overlaps with ADDRTYPE

### WEATHER

In [50]:
df.groupby(['WEATHER','SEVERITYCODE']).size()

WEATHER                   SEVERITYCODE
Blowing Sand/Dirt         1                  41
                          2                  15
Clear                     1               75295
                          2               35840
Fog/Smog/Smoke            1                 382
                          2                 187
Other                     1                 716
                          2                 116
Overcast                  1               18969
                          2                8745
Partly Cloudy             1                   2
                          2                   3
Raining                   1               21969
                          2               11176
Severe Crosswind          1                  18
                          2                   7
Sleet/Hail/Freezing Rain  1                  85
                          2                  28
Snowing                   1                 736
                          2                 171
U

In [51]:
df.groupby('WEATHER').agg({'SEVERITYCODE': ['count', 'mean', 'std']})

SEVERITYCODE                    
                                count      mean       std
WEATHER                                                  
Blowing Sand/Dirt                  56  1.267857  0.446850
Clear                          111135  1.322491  0.467432
Fog/Smog/Smoke                    569  1.328647  0.470135
Other                             832  1.139423  0.346596
Overcast                        27714  1.315544  0.464741
Partly Cloudy                       5  1.600000  0.547723
Raining                         33145  1.337185  0.472756
Severe Crosswind                   25  1.280000  0.458258
Sleet/Hail/Freezing Rain          113  1.247788  0.433651
Snowing                           907  1.188534  0.391353
Unknown                         15091  1.054072  0.226167

DECISION: WEATHER will be used in the model as there appears to be enough variation across the different weather conditions that it may be useful.

### ROADCOND

In [52]:
df.groupby(['ROADCOND','SEVERITYCODE']).size()

ROADCOND        SEVERITYCODE
Dry             1               84446
                2               40064
Ice             1                 936
                2                 273
Oil             1                  40
                2                  24
Other           1                  89
                2                  43
Sand/Mud/Dirt   1                  52
                2                  23
Snow/Slush      1                 837
                2                 167
Standing Water  1                  85
                2                  30
Unknown         1               14329
                2                 749
Wet             1               31719
                2               15755
dtype: int64

In [53]:
df.groupby('ROADCOND').agg({'SEVERITYCODE': ['count', 'mean', 'std']})

SEVERITYCODE                    
                      count      mean       std
ROADCOND                                       
Dry                  124510  1.321773  0.467158
Ice                    1209  1.225806  0.418285
Oil                      64  1.375000  0.487950
Other                   132  1.325758  0.470443
Sand/Mud/Dirt            75  1.306667  0.464215
Snow/Slush             1004  1.166335  0.372566
Standing Water          115  1.260870  0.441031
Unknown               15078  1.049675  0.217280
Wet                   47474  1.331866  0.470888

DECISION: ROADCOND will be used in the model as there appears to be enough variation across the different road conditions that it may be useful.

### LIGHTCOND

In [54]:
df.groupby(['LIGHTCOND','SEVERITYCODE']).size()

LIGHTCOND                 SEVERITYCODE
Dark - No Street Lights   1                1203
                          2                 334
Dark - Street Lights Off  1                 883
                          2                 316
Dark - Street Lights On   1               34032
                          2               14475
Dark - Unknown Lighting   1                   7
                          2                   4
Dawn                      1                1678
                          2                 824
Daylight                  1               77593
                          2               38544
Dusk                      1                3958
                          2                1944
Other                     1                 183
                          2                  52
Unknown                   1               12868
                          2                 605
dtype: int64

In [55]:
df.groupby('LIGHTCOND').agg({'SEVERITYCODE': ['count', 'mean', 'std']})

SEVERITYCODE                    
                                count      mean       std
LIGHTCOND                                                
Dark - No Street Lights          1537  1.217306  0.412547
Dark - Street Lights Off         1199  1.263553  0.440743
Dark - Street Lights On         48507  1.298411  0.457565
Dark - Unknown Lighting            11  1.363636  0.504525
Dawn                             2502  1.329337  0.470066
Daylight                       116137  1.331884  0.470892
Dusk                             5902  1.329380  0.470028
Other                             235  1.221277  0.415992
Unknown                         13473  1.044905  0.207102

DECISION: LIGHTCOND will be used in the model as there appears to be enough variation across the different light conditions that it may be useful.